In [19]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import re
from pathlib import Path

# Paths
PANEL_CSV = "../data/panels/company_weekly_panel_analysis_ready.csv"
DERIVED = Path("/Users/beszabo/bene/szakdolgozat/data/derived")

# Load panel
panel = pd.read_csv(PANEL_CSV, parse_dates=["week_start"])  # expects columns created earlier
panel["week_fe"] = panel["week_start"].dt.strftime("%Y-%W")  # weekly FE key

# Global weekly Reddit activity proxy (total memes per week across all brands) and its lag
weekly_reddit_activity = (
    panel.groupby("week_start")["num_memes"].sum().sort_index()
)
panel["reddit_activity"] = panel["week_start"].map(weekly_reddit_activity)
panel["reddit_activity_L1"] = panel["week_start"].map(weekly_reddit_activity.shift(1))


# TWFE OLS with clustered SEs

In [20]:

def run_twfe(
    df: pd.DataFrame,
    outcome: str,
    include_current: bool = True,
    lags: list[int] = [1, 2, 3, 4],
):
    # Select predictors that exist in the dataframe
    mention_vars = [f"NYT_mention_L{k}" for k in lags if f"NYT_mention_L{k}" in df.columns]
    sent_vars = [f"nyt_sentiment_L{k}" for k in lags if f"nyt_sentiment_L{k}" in df.columns]
    if include_current:
        if "NYT_mention" in df.columns:
            mention_vars = ["NYT_mention"] + mention_vars
        if "nyt_sentiment" in df.columns:
            sent_vars = ["nyt_sentiment"] + sent_vars
    control_vars = []
    if "reddit_activity_L1" in df.columns:
        control_vars = ["reddit_activity_L1"]
    X = mention_vars + sent_vars + control_vars

    needed = [outcome, "company", "week_fe"] + X
    d = df.dropna(subset=needed).copy()
    if d.empty:
        print(f"[WARN] No rows left after dropping NaNs for {outcome}. Skipping.")
        return None

    formula = f"{outcome} ~ " + " + ".join(X + ["C(company)", "C(week_fe)"])
    print("\n===", outcome, "===")
    print("N before drop:", len(df), " N after drop:", len(d))
    print("Formula:", formula)

    model = smf.ols(formula, data=d).fit(cov_type="cluster", cov_kwds={"groups": d["company"]})
    # print(model.summary())

    # Cumulative lag tests (only over lag terms, not including current)
    m_lags = [f"NYT_mention_L{k}" for k in lags if f"NYT_mention_L{k}" in d.columns]
    s_lags = [f"nyt_sentiment_L{k}" for k in lags if f"nyt_sentiment_L{k}" in d.columns]
    if m_lags:
        test_str = " + ".join(m_lags) + " = 0"
        # print("\nSum of NYT_mention lags (L1..L4) == 0:")
        # print(model.t_test(test_str))
    if s_lags:
        test_str = " + ".join(s_lags) + " = 0"
        # print("\nSum of nyt_sentiment lags (L1..L4) == 0:")
        # print(model.t_test(test_str))

    return model

In [22]:

# Collect key coefficients (current + L1..L4) for plotting lag effects later
def collect_key_coefs(model, outcome_name: str):
    if model is None:
        return []
    rows = []
    params = model.params
    ses = model.bse
    pvals = model.pvalues

    for term in params.index:
        # match NYT_mention, NYT_mention_L#, nyt_sentiment, nyt_sentiment_L#
        m = re.fullmatch(r"(NYT_mention)(?:_L(\d+))?", term)
        s = re.fullmatch(r"(nyt_sentiment)(?:_L(\d+))?", term)
        if m:
            predictor = "NYT_mention"
            lag = int(m.group(2)) if m.group(2) else 0
        elif s:
            predictor = "nyt_sentiment"
            lag = int(s.group(2)) if s.group(2) else 0
        else:
            continue
        rows.append({
            "outcome": outcome_name,
            "predictor": predictor,
            "lag": lag,
            "coef": float(params[term]),
            "se": float(ses[term]),
            "pval": float(pvals[term]),
        })
    return rows


## Mention-only TWFE models (no sentiment terms), brand and week fixed effects remain

In [23]:

def run_twfe_mention_only(
    df: pd.DataFrame,
    outcome: str,
    include_current: bool = True,
    lags: list[int] = [1, 2, 3, 4],
):
    mention_vars = [f"NYT_mention_L{k}" for k in lags if f"NYT_mention_L{k}" in df.columns]
    if include_current and "NYT_mention" in df.columns:
        mention_vars = ["NYT_mention"] + mention_vars
    control_vars = []
    if "reddit_activity_L1" in df.columns:
        control_vars = ["reddit_activity_L1"]
    X = mention_vars + control_vars

    needed = [outcome, "company", "week_fe"] + X
    d = df.dropna(subset=needed).copy()
    if d.empty:
        print(f"[WARN] No rows left after dropping NaNs for {outcome} (mention-only). Skipping.")
        return None

    formula = f"{outcome} ~ " + " + ".join(X + ["C(company)", "C(week_fe)"])
    print("\n=== Mention-only:", outcome, "===")
    print("N before drop:", len(df), " N after drop:", len(d))
    print("Formula:", formula)

    model = smf.ols(formula, data=d).fit(cov_type="cluster", cov_kwds={"groups": d["company"]})
    print(model.summary())
    
    # Cumulative lag test for mentions
    m_lags = [f"NYT_mention_L{k}" for k in lags if f"NYT_mention_L{k}" in d.columns]
    if m_lags:
        test_str = " + ".join(m_lags) + " = 0"
        print("\nSum of NYT_mention lags (L1..L4) == 0:")
        print(model.t_test(test_str))

    return model




In [16]:
def export_coefficients(
    df: pd.DataFrame,
    model_type: str,
    outcomes=("log1p_meme_volume", "mean_meme_sentiment", "log1p_meme_engagement"),
    lag_sets=([1, 2, 3, 4], [1, 2, 3], [1, 2], [1]),
    include_current: bool = True,
):
    results = {}
    for lags in lag_sets:
        run_fn = run_twfe_mention_only if model_type == "mention_only" else run_twfe
        models = {
            outcome: run_fn(df, outcome=outcome, include_current=include_current, lags=lags)
            for outcome in outcomes
        }
        
        rows = []
        for outcome_name, model in models.items():
            rows += collect_key_coefs(model, outcome_name)
        coefs = pd.DataFrame(rows).sort_values(["outcome", "predictor", "lag"]).reset_index(drop=True)
        suffix = f"{len(lags)}_lags"
        name_stub = "mention_only" if model_type == "mention_only" else "with_sentiment"
        out_path = DERIVED / f"twfe_key_coefficients_{name_stub}_{suffix}.csv"
        coefs.to_csv(out_path, index=False)
        print(f"Saved key coefficients to: {out_path}")
        print(coefs.head(12).to_string(index=False))
        results[suffix] = coefs
    return results

In [24]:
# Execute and export
_ = export_coefficients(panel, model_type="mention_only", outcomes=("log1p_meme_volume", "mean_meme_sentiment", "log1p_meme_engagement"))
_ = export_coefficients(panel, model_type="sentiment_including", outcomes=("log1p_meme_volume", "log1p_meme_engagement"))



=== Mention-only: log1p_meme_volume ===
N before drop: 8904  N after drop: 8568
Formula: log1p_meme_volume ~ NYT_mention + NYT_mention_L1 + NYT_mention_L2 + NYT_mention_L3 + NYT_mention_L4 + reddit_activity_L1 + C(company) + C(week_fe)
                            OLS Regression Results                            
Dep. Variable:      log1p_meme_volume   R-squared:                       0.418
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                 1.750e+11
Date:                Sun, 16 Nov 2025   Prob (F-statistic):               0.00
Time:                        16:59:20   Log-Likelihood:                -1931.7
No. Observations:                8568   AIC:                             4243.
Df Residuals:                    8378   BIC:                             5584.
Df Model:                         189                                         
Covariance Type:              cluster               

/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 190, but rank is 83
  warnings.warn('covariance of constraints does not have full '
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 175, but rank is 58
  warnings.warn('covariance of constraints does not have full '


                             OLS Regression Results                            
Dep. Variable:     mean_meme_sentiment   R-squared:                       0.263
Model:                             OLS   Adj. R-squared:                  0.101
Method:                  Least Squares   F-statistic:                -8.394e+09
Date:                 Sun, 16 Nov 2025   Prob (F-statistic):               1.00
Time:                         16:59:20   Log-Likelihood:                 566.23
No. Observations:                  971   AIC:                            -782.5
Df Residuals:                      796   BIC:                             71.25
Df Model:                          174                                         
Covariance Type:               cluster                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Inte

/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 190, but rank is 83
  warnings.warn('covariance of constraints does not have full '
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 190, but rank is 83
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:      log1p_meme_volume   R-squared:                       0.417
Model:                            OLS   Adj. R-squared:                  0.404
Method:                 Least Squares   F-statistic:                 2.653e+10
Date:                Sun, 16 Nov 2025   Prob (F-statistic):               0.00
Time:                        16:59:21   Log-Likelihood:                -1928.3
No. Observations:                8652   AIC:                             4237.
Df Residuals:                    8462   BIC:                             5579.
Df Model:                         189                                         
Covariance Type:              cluster                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

...

                             OLS Regression Results                            
Dep. Variable:     mean_meme_sentiment   R-squared:                       0.261
Model:                             OLS   Adj. R-squared:                  0.101
Method:                  Least Squares   F-statistic:                -1.451e+10
Date:                 Sun, 16 Nov 2025   Prob (F-statistic):               1.00
Time:                         16:59:21   Log-Likelihood:                 565.68
No. Observations:                  977   AIC:                            -781.4
Df Residuals:                      802   BIC:                             73.42
Df Model:                          174                                         
Covariance Type:               cluster                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Inte

/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 190, but rank is 83
  warnings.warn('covariance of constraints does not have full '
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 190, but rank is 83
  warnings.warn('covariance of constraints does not have full '
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 175, but rank is 58
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:      log1p_meme_volume   R-squared:                       0.415
Model:                            OLS   Adj. R-squared:                  0.402
Method:                 Least Squares   F-statistic:                 2.338e+11
Date:                Sun, 16 Nov 2025   Prob (F-statistic):               0.00
Time:                        16:59:22   Log-Likelihood:                -1919.8
No. Observations:                8820   AIC:                             4220.
Df Residuals:                    8630   BIC:                             5566.
Df Model:                         189                                         
Covariance Type:              cluster                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 190, but rank is 83
  warnings.warn('covariance of constraints does not have full '


                              OLS Regression Results                             
Dep. Variable:     log1p_meme_engagement   R-squared:                       0.318
Model:                               OLS   Adj. R-squared:                  0.303
Method:                    Least Squares   F-statistic:                 2.008e+04
Date:                   Sun, 16 Nov 2025   Prob (F-statistic):          1.15e-155
Time:                           16:59:22   Log-Likelihood:                -1852.5
No. Observations:                   8820   AIC:                             4085.
Df Residuals:                       8630   BIC:                             5431.
Df Model:                            189                                         
Covariance Type:                 cluster                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------

# Coefficient-lag spline

In [27]:
# Coefficient–lag plots (with sentiment vs mention-only)
import matplotlib.pyplot as plt
from pathlib import Path

OUT_DIR = Path("/Users/beszabo/bene/szakdolgozat/figures/results/lead_lag")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Build coefficient DataFrames from fitted models
def build_coef_df_with():
    rows = []
    for outcome_name in [
        "log1p_meme_volume",
        "mean_meme_sentiment",
        "log1p_meme_engagement",
    ]:
        model = run_twfe(panel, outcome=outcome_name, include_current=True)
        nobs = int(model.nobs) if model is not None else 0
        r = collect_key_coefs(model, outcome_name)
        for row in r:
            row["nobs"] = nobs
        rows += r
    dfw = pd.DataFrame(rows)
    if not dfw.empty:
        dfw["ci_low"] = dfw["coef"] - 1.96 * dfw["se"]
        dfw["ci_high"] = dfw["coef"] + 1.96 * dfw["se"]
    return dfw


def build_coef_df_mention_only():
    models = [
        (run_twfe_mention_only(panel, outcome="log1p_meme_volume", include_current=True), "log1p_meme_volume"),
        (run_twfe_mention_only(panel, outcome="mean_meme_sentiment", include_current=True), "mean_meme_sentiment"),
        (run_twfe_mention_only(panel, outcome="log1p_meme_engagement", include_current=True), "log1p_meme_engagement"),
    ]
    rows = []
    for model, outcome_name in models:
        nobs = int(model.nobs) if model is not None else 0
        r = collect_key_coefs(model, outcome_name)
        for row in r:
            row["nobs"] = nobs
        rows += r
    dfm = pd.DataFrame(rows)
    if not dfm.empty:
        dfm["ci_low"] = dfm["coef"] - 1.96 * dfm["se"]
        dfm["ci_high"] = dfm["coef"] + 1.96 * dfm["se"]
    return dfm


def plot_outcome(df: pd.DataFrame, outcome: str, predictors: list[str], title: str, out_file: Path):
    sub = df[df["outcome"] == outcome]
    nobs = None
    if not sub.empty and "nobs" in sub.columns and sub["nobs"].notna().any():
        try:
            nobs = int(sub["nobs"].iloc[0])
        except Exception:
            nobs = None
    plt.figure(figsize=(6, 4))
    for pred in predictors:
        s = sub[sub["predictor"] == pred].sort_values("lag")
        if s.empty:
            continue
        yerr = 1.96 * s["se"].values
        plt.errorbar(s["lag"], s["coef"], yerr=yerr, fmt="o-", capsize=3, label=pred)
    plt.axhline(0, color="black", linewidth=1)
    plt.xlabel("Lag k (weeks)")
    plt.ylabel("Coefficient")
    plt.title(f"{title} (N={nobs})" if nobs is not None else title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_file, dpi=200)
    plt.close()


# Generate and save plots
with_df = build_coef_df_with()
mo_df = build_coef_df_mention_only()

outcomes = ["log1p_meme_volume", "mean_meme_sentiment", "log1p_meme_engagement"]
for outcome in outcomes:
    # With sentiment: plot both NYT_mention and nyt_sentiment if present
    preds_with = [p for p in ["NYT_mention", "nyt_sentiment"] if (not with_df.empty and ((with_df["outcome"] == outcome) & (with_df["predictor"] == p)).any())]
    if preds_with:
        path_with = OUT_DIR / f"coef_paths_with_sentiment_{outcome}.png"
        plot_outcome(with_df, outcome, preds_with, f"With sentiment — {outcome}", path_with)
        print(f"Saved: {path_with}")

    # Mention-only: only NYT_mention
    preds_mo = ["NYT_mention"] if (not mo_df.empty and ((mo_df["outcome"] == outcome) & (mo_df["predictor"] == "NYT_mention")).any()) else []
    if preds_mo:
        path_mo = OUT_DIR / f"coef_paths_mention_only_{outcome}.png"
        plot_outcome(mo_df, outcome, preds_mo, f"Mention-only — {outcome}", path_mo)
        print(f"Saved: {path_mo}")



=== log1p_meme_volume ===
N before drop: 8904  N after drop: 2092
Formula: log1p_meme_volume ~ NYT_mention + NYT_mention_L1 + NYT_mention_L2 + NYT_mention_L3 + NYT_mention_L4 + nyt_sentiment + nyt_sentiment_L1 + nyt_sentiment_L2 + nyt_sentiment_L3 + nyt_sentiment_L4 + reddit_activity_L1 + C(company) + C(week_fe)

=== mean_meme_sentiment ===
N before drop: 8904  N after drop: 400
Formula: mean_meme_sentiment ~ NYT_mention + NYT_mention_L1 + NYT_mention_L2 + NYT_mention_L3 + NYT_mention_L4 + nyt_sentiment + nyt_sentiment_L1 + nyt_sentiment_L2 + nyt_sentiment_L3 + nyt_sentiment_L4 + reddit_activity_L1 + C(company) + C(week_fe)

=== log1p_meme_engagement ===
N before drop: 8904  N after drop: 2092
Formula: log1p_meme_engagement ~ NYT_mention + NYT_mention_L1 + NYT_mention_L2 + NYT_mention_L3 + NYT_mention_L4 + nyt_sentiment + nyt_sentiment_L1 + nyt_sentiment_L2 + nyt_sentiment_L3 + nyt_sentiment_L4 + reddit_activity_L1 + C(company) + C(week_fe)

=== Mention-only: log1p_meme_volume ===
N b

/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 190, but rank is 83
  warnings.warn('covariance of constraints does not have full '
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 175, but rank is 58
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:      log1p_meme_volume   R-squared:                       0.418
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                 1.750e+11
Date:                Sun, 16 Nov 2025   Prob (F-statistic):               0.00
Time:                        17:18:46   Log-Likelihood:                -1931.7
No. Observations:                8568   AIC:                             4243.
Df Residuals:                    8378   BIC:                             5584.
Df Model:                         189                                         
Covariance Type:              cluster                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 190, but rank is 83
  warnings.warn('covariance of constraints does not have full '


                              OLS Regression Results                             
Dep. Variable:     log1p_meme_engagement   R-squared:                       0.318
Model:                               OLS   Adj. R-squared:                  0.302
Method:                    Least Squares   F-statistic:                 4.491e+11
Date:                   Sun, 16 Nov 2025   Prob (F-statistic):               0.00
Time:                           17:18:47   Log-Likelihood:                -1834.2
No. Observations:                   8568   AIC:                             4048.
Df Residuals:                       8378   BIC:                             5389.
Df Model:                            189                                         
Covariance Type:                 cluster                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------